In [7]:
import json
import os
import argparse
import dotenv

from dotenv import load_dotenv
from openai import OpenAI

from tqdm import tqdm

load_dotenv()

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

questions = "datasets/spider_data/dev.json"

system_prompt = """You are an expert SQL assistant specialized in converting natural language queries into accurate SQL statements. You:
1. Understand database schemas and relationships
2. Generate standard SQL queries that follow best practices
3. Consider edge cases and data validation
4. Can handle complex joins, aggregations, and nested queries
5. Provide explanations for the generated SQL when needed

When given a question, you will convert it to a valid SQL query based on the provided database schema."""


# Load questions from JSON file
with open(questions, 'r') as f:
    questions = json.load(f)

print ("total number of questions: ", len(questions))

total number of questions:  1034


In [8]:
from utils.spider import load_tables
import sqlite3
from pathlib import Path



In [10]:
spider_table = 'tables.json'
spider_dir = "datasets/spider_data"
db = "database"

dataset_dir = spider_dir
table = spider_table

# load schemas
schemas, _ = load_tables([os.path.join(dataset_dir, table)])

In [11]:
#Backup in-memory copies of all the DBs and create the live connections
for db_id, schema in tqdm(schemas.items(), desc="DB connections"):
    sqlite_path = Path(dataset_dir) / db / db_id / f"{db_id}.sqlite"
    source: sqlite3.Connection
    with sqlite3.connect(str(sqlite_path)) as source:
        dest = sqlite3.connect(':memory:')
        dest.row_factory = sqlite3.Row
        source.backup(dest)
    schema.connection = dest


DB connections: 100%|██████████| 166/166 [00:02<00:00, 81.90it/s]


In [23]:
db_id = "allergy_1"  # or any other db_id from your schemas
connection = schemas[db_id].connection
cursor = connection.cursor()

# Query sqlite_master table to get all CREATE statements
cursor.execute("""
    SELECT sql 
    FROM sqlite_master 
    WHERE type='table' AND sql IS NOT NULL
""")

# Print each CREATE statement
for (sql,) in cursor.fetchall():
    print(sql)
    #print("\n" + "-"*50 + "\n")

CREATE TABLE Allergy_Type (
       Allergy 		  VARCHAR(20) PRIMARY KEY,
       AllergyType 	  VARCHAR(20)
)
CREATE TABLE Has_Allergy (
       StuID 		 INTEGER,
       Allergy 		 VARCHAR(20),
       FOREIGN KEY(StuID) REFERENCES Student(StuID),
       FOREIGN KEY(Allergy) REFERENCES Allergy_Type(Allergy)
)
CREATE TABLE Student (
        StuID        INTEGER PRIMARY KEY,
        LName        VARCHAR(12),
        Fname        VARCHAR(12),
        Age      INTEGER,
        Sex      VARCHAR(1),
        Major        INTEGER,
        Advisor      INTEGER,
        city_code    VARCHAR(3)
 )


In [21]:
display_pdf